In [1]:
import pandas as pd
import re

In [2]:
conserved_ptm_map_df = pd.read_csv('Data/Mapped_PTMs.csv')
conserved_ptm_map_df

,Unnamed: 0,Organism,Gene,Peptide,Observed Modifications,best_locs,start_position,end_position,a_start_position,a_end_position,alignment_length,fasta_id,a_best_locs_positions
0,0,Bacillus_subtilis,accB,AIDESTIDEFVYENEGVSLK,Cation:Fe[III](52.911464),AIDestideFVYENEGVSLK,12,31,39,58,195,Bacillus_subtilis(accB),"[42, 43, 44, 45, 46, 47]"
1,1,Stigmatella_aurantiaca,accB,RGHGPAPTIVHAAPVSPSVSPAPAVEYAAPAPAR,Cation:Fe[III](52.911464),RGHGPAPTIVHAAPVSpsvspapaveYAAPAPAR,61,94,62,95,195,Stigmatella_aurantiaca(accB),"[78, 79, 80, 81, 82, 83, 84, 85, 86, 87]"
2,2,Chryseobacterium_indologenes,accB,ILVDDATPVEYDQPLFLVDPS,Ammonium(17.026549),ILVDDATPVEYDQplfLVDPS,139,159,174,194,195,Chryseobacterium_indologenes(accB),"[187, 188, 189]"
3,3,Chryseobacterium_indologenes,accB,ILVDDATPVEYDQPLFLVDPS,Ammonium(17.026549),ILVDDATPVEYDQPLFLvDPS,139,159,174,194,195,Chryseobacterium_indologenes(accB),[191]
4,4,Citrobacter_freundii,accB,AILVESGQPVEFDEPLVVIE,Ammonium(17.026549),AILVESGQPVEFDEPLVVIE,136,155,173,192,195,Citrobacter_freundii(accB),[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15050,15050,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl(14.015650),DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
15051,15051,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl(14.015650),DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
15052,15052,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl(14.015650),DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
15053,15053,Rhodopseudomonas_palustris,zwf,FANGMFEPIWNR,Carboxy->Thiocarboxy(15.977156),FaNGMFEPIWNR,205,216,234,245,557,Rhodopseudomonas_palustris(zwf),[235]


In [6]:
# range of conserved PTMs

conserved_ptm_rows = []
for gene in pd.unique(conserved_ptm_map_df.Gene):
    gene_df = conserved_ptm_map_df[conserved_ptm_map_df.Gene == gene]
    gene_length = list(gene_df.alignment_length)[0]
    for modification in pd.unique(gene_df['Observed Modifications']):
        conserved_ptms = {}
        modifications = {}
        mod_df = gene_df[gene_df['Observed Modifications'] == modification]
        for i in range(0, gene_length + 1):
            modifications[i] =  [set(), set()]
        for index, row in mod_df.iterrows():
            organism = row['Organism']
            fasta_id = row['fasta_id']
            a_start = row['a_start_position']
            a_end = row['a_end_position'] + 1
            for i in range(a_start, a_end):
                modifications[i][0].add(organism)
                modifications[i][1].add(fasta_id)
        for i in range(0, len(modifications)):
            if len(modifications[i][0]) < 2:
                del modifications[i]
        if len(modifications) > 0:
            conserved_ptms['Gene'] = gene
            conserved_ptms['Modification'] = modification
            conserved_ptms['mod_residues_dic'] = modifications
            conserved_ptm_rows.append(conserved_ptms)
conserved_ptms = pd.DataFrame(conserved_ptm_rows)
conserved_ptms.to_csv('Data/Conserved_ptms.csv')
conserved_ptms      
            
            

,Gene,Modification,mod_residues_dic
0,accB,Ammonium(17.026549),"{174: [{'Chryseobacterium_indologenes', 'Citro..."
1,ackA,Carbamyl(43.005814),"{400: [{'Cellvibrio_gilvus', 'Citrobacter_freu..."
2,acnA,"Cation:Na(21.981943), glyoxalAGE(21.984350)","{271: [{'Sulfobacillus_thermosulfidooxidans', ..."
3,acnA,Carbamyl(43.005814),"{519: [{'Mycobacterium_smegmatis', 'Sulfobacil..."
4,acpP,Carbamyl(43.005814),"{82: [{'Cellvibrio_gilvus', 'Sulfobacillus_the..."
...,...,...,...
746,tyrS,Ammonia-loss(-17.026549),"{422: [{'Delftia_acidovorans', 'Ruminococcus_g..."
747,tyrS,Methyl(14.015650),"{368: [{'Rhodopseudomonas_palustris', 'Cupriav..."
748,valS,"Ammonium(17.026549), Methyl:2H(3)(17.034480)","{1153: [{'Bacillus_subtilis', 'Coprococcus_com..."
749,ychF,Ammonia-loss(-17.026549),"{263: [{'Bacillus_subtilis', 'Cellulophaga_bal..."


In [7]:
conserved_ptm_rows = []
for gene in pd.unique(conserved_ptm_map_df.Gene):
    gene_df = conserved_ptm_map_df[conserved_ptm_map_df.Gene == gene]
    gene_length = list(gene_df.alignment_length)[0]
    for modification in pd.unique(gene_df['Observed Modifications']):
        conserved_ptms = {}
        modifications = {}
        mod_df = gene_df[gene_df['Observed Modifications'] == modification]
        for i in range(0, gene_length + 1):
            modifications[i] =  [set(), set()]
        for index, row in mod_df.iterrows():
            organism = row['Organism']
            a_start = row['a_start_position']
            a_end = row['a_end_position'] + 1
            fasta_id  = row['fasta_id']
            # convert string to list
            a_best_locs = []
            best_locs = row['a_best_locs_positions']
            for i in re.split(',', best_locs):
                i = i.replace('[', '')
                i = i.replace(']', '')
                i = i.replace(' ', '')
                if i == '': # no best loc specified
                    a_best_locs = range(a_start, a_end)
                else:
                    i = int(i)
                    a_best_locs.append(i)
            for i in a_best_locs:
                modifications[i][0].add(organism)
                modifications[i][1].add(fasta_id)
        for i in range(0, len(modifications)):
            if len(modifications[i][0]) < 2:
                del modifications[i]
        if len(modifications) > 0:
            conserved_ptms['Gene'] = gene
            conserved_ptms['Modification'] = modification
            conserved_ptms['mod_residues_dic'] = modifications
            conserved_ptm_rows.append(conserved_ptms)
conserved_ptms = pd.DataFrame(conserved_ptm_rows)
conserved_ptms.to_csv('Data/Conserved_ptms_best_loc.csv')
conserved_ptms

,Gene,Modification,mod_residues_dic
0,accB,Ammonium(17.026549),"{187: [{'Chryseobacterium_indologenes', 'Citro..."
1,ackA,Carbamyl(43.005814),"{403: [{'Cellvibrio_gilvus', 'Citrobacter_freu..."
2,acnA,Carbamyl(43.005814),"{522: [{'Mycobacterium_smegmatis', 'Sulfobacil..."
3,acpP,Carbamyl(43.005814),"{94: [{'Delftia_acidovorans', 'Acidiphilium_cr..."
4,acpP,"Acetyl:13C(2)(44.017274), Delta:H(4)C(2)O(-1)S...","{155: [{'Delftia_acidovorans', 'Stigmatella_au..."
...,...,...,...
562,typA,"Ammonium(17.026549), Methyl:2H(3)(17.034480)","{112: [{'Citrobacter_freundii', 'Coprococcus_c..."
563,tyrS,Methyl(14.015650),"{392: [{'Rhodopseudomonas_palustris', 'Cupriav..."
564,valS,"Ammonium(17.026549), Methyl:2H(3)(17.034480)","{1159: [{'Bacillus_subtilis', 'Coprococcus_com..."
565,ychF,Ammonia-loss(-17.026549),"{263: [{'Bacillus_subtilis', 'Cellulophaga_bal..."
